# **Biblioteki**

Biblioteki użyte w implementacji.


In [ ]:
!pip install mahotas
import mahotas
from scipy.spatial import distance as dist
import matplotlib.pyplot as plt
import numpy as np
import argparse
import glob
import cv2
from google.colab.patches import cv2_imshow
import imutils

from math import sqrt
from skimage.transform import resize

# **Klasa Timer**

Klasa służąca do pomiarów czasu

In [3]:
import time

class Timer:
    def __init__(self):
        self._start = None
        self._elapsed = None

    def start(self):
        
        if self._start is not None:
            raise Exception(f"Timer is running. Use .stop() to stop it")
        
        print("TIMER: Czas start")
        self._start = time.perf_counter()

    def stop(self):
        
        if self._start is None:
            raise Exception(f"Timer is not running. Use .start() to start it")

        self._elapsed = time.perf_counter() - self._start
        print("TIMER: Czas stop")
        self._start = None


    def cleanTimer(self):
      self._start = None
      self._elapsed = None

    def printElapsedTime(self):
      print(f"\tElapsed time: {self._elapsed:0.4f} seconds")

# ***Klasa ImageData***


In [6]:
class ImageData:
    def __init__(self, kind, path):
        self._kind = kind
        self._path = path
        self._name = path[path.rfind("/") + 1:]
        
        image = cv2.imread (path)       
        self._image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        self._normalizeImage = cv2.resize(self._image, (2**10,2**10), interpolation = cv2.INTER_AREA)
        self._greyImage = cv2.cvtColor(self._normalizeImage, cv2.COLOR_BGR2GRAY) 
        self._matrix = cv2.resize(self._greyImage, (8, 8), interpolation = cv2.INTER_AREA)
        self._historgramMatrix = cv2.calcHist([self._greyImage],[0],None,[16],[0,256])
        self._historgramByMedium = {}
        self._key = []
        self._desc = []

    def printName(self):
      print(self._name)
    
    def printPath(self):
      print(self._path)

    def printKind(self):
      print(self._kind)

    def printMatrix(self):
      print(" === " + self._name + " ===\n " + str(self._matrix))

    def getKind(self):
      return self._kind
    
    def getImage(self):
      return self._image
    
    def getGreyImage(self):
      return self._greyImage

    def getNormalizeImage(self):
      return self._normalizeImage
    
    def getMatrix(self):
      return self._matrix

    def getHistorgramMatrix(self):
      return self._historgramMatrix    

    def setHistorgramByMedium(self, hist):
      self._historgramByMedium = hist
      self._epsilon = abs(self._historgramByMedium["less"] - self._historgramByMedium["more"])

    def printStatus(self):
      print(self._name + " | " + str(self._historgramByMedium["less"]) + " | " + str(self._historgramByMedium["more"]) + " | " + str(self._epsilon))

    def getEpsilon(self):
      return self._epsilon

    def setKeyDesc(self, key, desc):
      self._key = key
      self._desc = desc

    def getKeyDesc(self):
      return self._key, self._desc

    def setName(self, name):
      self._name = name

# ***Klasa ListImageData***

In [9]:
class ListImageData:
    def __init__(self, kind, pathList):
        self._kind = kind
        self._listImagesData = []

        for imagePath in pathList:
          imageData = ImageData(self._kind, imagePath)
          self._listImagesData.append(imageData)

    def printImagesNames(self):
      for image in self._listImagesData:
        image.printName()

    def printImagesPath(self):
      for image in self._listImagesData:
        image.printPath()

    def printImagesMatrix(self):
      for image in self._listImagesData:
        image.printMatrix()

    def getImagesMatrixList(self):
      matrixList = []
      
      for image in self._listImagesData:
        matrixList.append(image.getMatrix())
      
      return matrixList

    def getSize(self):
      return len(self._listImagesData)

    def getImagesData(self):
      return self._listImagesData

# ***Klasa ImagePrinter***

In [8]:
class ImagePrinter:
    def __init__(self, listImageData):
      self._listImageData = listImageData

    def showImageList(self):
      length = len(self._listImageData)
      row = 0
      if (length > 5 and length % 2 == 0):
        row = 2
        length = length/2

      figure, ax = plt.subplots(int(row), int(length), figsize=(16, 8)) 
      figure.tight_layout(pad=3.0)
      counter=0

      for i in range(int(row)):
        for j in range(int(length)):
          ax[i, j].set_title(self._listImageData[counter]._name)
          ax[i, j].axis('off')
          ax[i, j].imshow(self._listImageData[counter].getImage(), cmap = 'gray')
          counter+=1

    def showImgs(self):
      length = len(self._listImageData)
      figure, ax = plt.subplots(1, length, figsize=(16, 8)) 
      i=0

      for imageData in self._listImageData:
        ax[i].set_title(imageData._name)
        ax[i].axis('off')
        ax[i].imshow(imageData.getImage(), cmap = 'gray')
        i+=1

    def showNormalizeImageList(self):
      length = len(self._listImageData)
      figure, ax = plt.subplots(1, length, figsize=(16, 8)) 
      i=0

      for imageData in self._listImageData:
        ax[i].set_title(imageData._name)
        ax[i].axis('off')
        ax[i].imshow(imageData.getNormalizeImage(), cmap = 'gray')
        i+=1
    
    def showGreyImageList(self):
      length = len(self._listImageData)
      figure, ax = plt.subplots(1, length, figsize=(16, 8)) 
      i=0

      for imageData in self._listImageData:
        ax[i].set_title(imageData._name)
        ax[i].axis('off')
        ax[i].imshow(imageData.getGreyImage(), cmap = 'gray')
        i+=1

    def showMatrix(self):
      length = len(self._listImageData)
      figure, ax = plt.subplots(1, length, figsize=(16, 8)) 
      i=0

      for imageData in self._listImageData:
        ax[i].set_title(imageData._name)
        ax[i].axis('off')
        ax[i].imshow(imageData.getMatrix(), cmap = 'gray')
        i+=1

    def showHistogram(self):
      length = len(self._listImageData)
      figure, ax = plt.subplots(1, length, figsize=(16, 8)) 
      i=0

      for imageData in self._listImageData:
        ax[i].set_title(imageData._name)
        ax[i].hist(imageData.getMatrix().ravel(),16,[0,256]);
        i+=1

    def showHistogramByMedium(self):
      length = len(self._listImageData)
      figure, ax = plt.subplots(1, length, figsize=(16, 8)) 
      i=0

      for imageData in self._listImageData:
        ax[i].set_title(imageData._name)
        ax[i].bar(*zip(*imageData._historgramByMedium.items()))
        i+=1

    def showMatches(self, query):
      fig, ax = plt.subplots()
      ax.axis('off')
      ax.set_title(query._name)
      ax.imshow(query.getImage(), cmap = 'gray')
      self.showImageList()

# ***Konfiguracja***

In [32]:
DESC_MODEL = 1

if (DESC_MODEL == 1):
  Descriptor = "Akaze"
else:
  Descriptor = "Zernike"

USE_PRE_CLASSIFICATION = True

#SET DEBUG MODE
DEBUG = False;
PRINT_IMAGES_NAME = False;
PRINT_IMAGES_PATH = False;
SHOW_DATA = False;
ifPrintTwoGroup = False;
ifPrintThreeGroup = False;

#Ścieżki do katalogów
ListfilesAlpha = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE2/ALPHA/*.jpg");
ListfilesBeta  = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE2/BETA/*.jpg");

#Const
ALPHA_NAME = "Pierwszy zbiór zdjęć";
BETA_NAME = "Drugi zbiór zdjęć";
ALPHA_KIND = "Alpha";
BETA_KIND = "Beta";


# ***Zbiór Aplha***

In [12]:
alphaListImageData = ListImageData(ALPHA_KIND, ListfilesAlpha)
alphaImagePrinter = ImagePrinter(alphaListImageData.getImagesData());

if (SHOW_DATA):
  alphaImagePrinter.showImgs()
  alphaImagePrinter.showNormalizeImageList()
  alphaImagePrinter.showGreyImageList()
  alphaImagePrinter.showMatrix()
  alphaImagePrinter.showHistogram()



# ***Zbiór Beta***

In [13]:
betaListImageData = ListImageData(BETA_KIND, ListfilesBeta)
betaImagePrinter = ImagePrinter(betaListImageData.getImagesData());

if (SHOW_DATA):
  betaImagePrinter.showImageList()
  betaImagePrinter.showNormalizeImageList()
  betaImagePrinter.showGreyImageList()
  betaImagePrinter.showMatrix()
  betaImagePrinter.showHistogram()

# ***Klasa PreClassificator***

In [14]:
class PreClassificator:

  def __init__(self, imageListAlpha, imageListBeta, epsilon, ifPrintTwoGroup, ifPrintThreeGroup):
    self.ifPrintTwoGroup = ifPrintTwoGroup
    self.ifPrintThreeGroup = ifPrintThreeGroup
    self.imageListAlpha = imageListAlpha
    self.imageListBeta = imageListBeta
    self.rangeEpsilon = epsilon

    self.groupOne = []
    self.groupTwo = []
    self.groupThree = []

    print("\nWstępna klasyfikacja (podział na podzbiory)")
    print("\tWartość epsilon: {}".format(str(self.rangeEpsilon)))

  def countMedium(self):
    sumAlpha = self.countSum(self.imageListAlpha.getImagesMatrixList())
    sumBeta = self.countSum(self.imageListBeta.getImagesMatrixList())

    sum = sumAlpha + sumBeta
    count = (self.imageListAlpha.getSize() * 64) + (self.imageListBeta.getSize() * 64)
    
    print("\tWyliczenie progu podziału: {}".format(str(sum/count)))
    
    if (self.ifPrintTwoGroup):
      print("\tŚrednia: {}\n\t\tSum: {}\n\t\tcount: {}".format(str(sum/count),str(sum),str(count)))

    self.medium = sum/count
    
  def countSum(self, matrixList):
    sum = 0;
    for matrix in matrixList:
      sum += np.matrix(matrix).sum()
    return sum

  def genHistByMediumLists(self):
    self.genHistByMedium(self.imageListAlpha.getImagesData())
    self.genHistByMedium(self.imageListBeta.getImagesData())

  def genHistByMedium(self, imageDataList):
    for imageData in imageDataList:
      img = imageData.getMatrix()
      less = 0
      more = 0

      for line in img:
        for pix in line:
            if pix <=self.medium :
              less += 1
            else :
              more += 1

      histByMedium = {}
      histByMedium["less"]= less
      histByMedium["more"]= more
      imageData.setHistorgramByMedium(histByMedium)    

  def genGroup(self): 
    for imageData in self.imageListAlpha.getImagesData():
      if imageData._historgramByMedium["less"] >= imageData._historgramByMedium["more"]:
        self.groupOne.append(imageData)
      else :
        self.groupTwo.append(imageData)
    
    for imageData in self.imageListBeta.getImagesData():
      if imageData._historgramByMedium["less"] >= imageData._historgramByMedium["more"]:
        self.groupOne.append(imageData)
      else :
        self.groupTwo.append(imageData) 

  def genGroupTree(self):
    i = len(self.groupOne) -1
    while i > 0:
      if (self.groupOne[i]._epsilon <= self.rangeEpsilon):
        self.groupThree.append(self.groupOne[i])
        self.groupOne.remove(self.groupOne[i])  
      i -= 1

    i = len(self.groupTwo) -1
    while i > 0:
      if (self.groupTwo[i]._epsilon <= self.rangeEpsilon):
        self.groupThree.append(self.groupTwo[i])
        self.groupTwo.remove(self.groupTwo[i])  
      i -= 1

  def printTwoGroup(self):
    print("========================Group 1==========================")
    for imageData in self.groupOne:
      imageData.printStatus()
    print("========================Group 2==========================")
    for imageData in self.groupTwo:
      imageData.printStatus()
    print("======================== END   ==========================")

  def printThreeGroup(self):
    print("========================Group 1==========================")
    for imageData in self.groupOne:
      imageData.printStatus()
    print("========================Group 2==========================")
    for imageData in self.groupTwo:
      imageData.printStatus()
    print("========================Group 3==========================")
    for imageData in self.groupThree:
      imageData.printStatus()
    print("======================== END   ==========================")
    
  def run(self):
    self.countMedium();
    
    self.genHistByMediumLists()

    self.genGroup();
    if (self.ifPrintTwoGroup):
      self.printTwoGroup();

    self.genGroupTree()
    self.printGroupsStatus()

    if (self.ifPrintThreeGroup):
      self.printThreeGroup();

    return self.groupOne, self.groupTwo, self.groupThree


  def printGroupsStatus(self):
    print("\tPodzielono obrazy na 3 zbiory")
    print("\t\tGroupOne (less <= more) size: {}".format(str(len(self.groupOne))))
    print("\t\tGroupOne (less >  more) size: {}".format(str(len(self.groupTwo))))
    print("\t\tGroupOne (gen. Epsilon) size: {}".format(str(len(self.groupThree))))

# Klasa ***GeneretorDescriptor***

In [15]:
class GeneretorDescriptor:
  
  def __init__(self, imageListAlpha, imageListBeta, debug):
    self.debug = debug
    self.imageListAlpha = imageListAlpha
    self.imageListBeta = imageListBeta

  def run(self):
     sum = self.imageListAlpha.getSize() + self.imageListBeta.getSize()
     i = 0
     print("\nGenerowanie cech lokalnych obrazów")

     for imageData in self.imageListAlpha.getImagesData():
       key, desc = self.generateDescForImage(imageData.getNormalizeImage())
       imageData.setKeyDesc(key, desc)
       self.printStatus(i, sum)
       i +=1

     for imageData in self.imageListBeta.getImagesData():
       i +=1
       key, desc = self.generateDescForImage(imageData.getNormalizeImage())
       imageData.setKeyDesc(key, desc)
       self.printStatus(i, sum)

     return self.imageListAlpha, self.imageListBeta

  def generateDescForImage(self, image):
     detector = cv2.AKAZE_create()
     key_points, desc = detector.detectAndCompute(image, None)
     return key_points, desc;
  
  def printStatus(self, progres, sum):
     if self.debug and (progres%10 == 0 or progres == sum or progres == 0):
       pro = (progres*100)/sum
       print("\t" + "% .2f " % (pro) + "% \t" + str(progres) + "/" + str(sum))

# ***ZernikeGenereatorDescriptor***

In [16]:
class ZernikeGenereatorDescriptor:
  
  def __init__(self, imageListAlpha, imageListBeta, debug):
    self.debug = debug
    self.imageListAlpha = imageListAlpha
    self.imageListBeta = imageListBeta

  def run(self):
     sum = self.imageListAlpha.getSize() + self.imageListBeta.getSize()
     i = 0
     print("\nGenerowanie cech lokalnych obrazów")

     for imageData in self.imageListAlpha.getImagesData():
       desc = self.generateDescForImage(imageData.getNormalizeImage())
       imageData._desc = desc
       self.printStatus(i, sum)
       i +=1

     for imageData in self.imageListBeta.getImagesData():
       i +=1
       desc = self.generateDescForImage(imageData.getNormalizeImage())
       imageData._desc = desc
       self.printStatus(i, sum)

     return self.imageListAlpha, self.imageListBeta

  def generateDescForImage(self, image):
     image = cv2.copyMakeBorder(image, 15, 15, 15, 15, cv2.BORDER_CONSTANT, value=255)
     thresh = cv2.bitwise_not(image)
     thresh[thresh > 0] = 255
     edged = cv2.Canny(thresh, 30, 200)
     cv2.waitKey(0)
   
     outline = np.zeros(edged.shape, dtype = "uint8")
     cnts = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
     cnts = imutils.grab_contours(cnts)
     cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[0] 
     cv2.drawContours(outline, [cnts], -1, 255, -1)
     
     moments = mahotas.features.zernike_moments(outline, 21)
     return moments
  
  def printStatus(self, progres, sum):
     if self.debug and (progres%10 == 0 or progres == sum or progres == 0):
       pro = (progres*100)/sum
       print("\t" + "% .2f " % (pro) + "% \t" + str(progres) + "/" + str(sum))

# ***Klasa ImageMatcher***

In [17]:
class ImageMatcher:

  def __init__(self, groupOne, groupTwo, groupThree, debug):
    self.debug = debug
    self.groupOne = groupOne
    self.groupTwo = groupTwo
    self.groupThree = groupThree
    self.matches = []

  def run(self, debug):
    print("\nPorównywanie obrazów")
    countAllOperation = self.compareDescImageList(debug, self.groupOne)
    countAllOperation += self.compareDescImageList(debug, self.groupTwo)
    countAllOperation +=self.compareDescImageList(debug, self.groupThree)
    
    print("\t|\tIlość wszystkich operacji: " + str(countAllOperation))
    return self.matches

  def genGroupToCompare(self, group):
    listAlpha = []
    listBeta = []

    for imageData in group:

      if imageData.getKind() == "Alpha":
        listAlpha.append(imageData)

      if imageData.getKind() == "Beta":
        listBeta.append(imageData)

    return listAlpha, listBeta 

  def compareDescImageList(self, debug, group):
    countOperation = 0;
    listAlpha, listBeta = self.genGroupToCompare(group)
    for imageDataAlpha in listAlpha:
      for imageDataBeta in listBeta:
          self.compareDescImage(imageDataAlpha, imageDataBeta, debug)
          countOperation = countOperation + 1;

    print("\t|\tIlość operacji: " + str(countOperation))
    return countOperation


  def compareDescImage(self, imageDataAlpha, imageDataBeta, debug):
    homography = 7.6285898e-01
    matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_BRUTEFORCE_HAMMING)
    nn_matches = matcher.knnMatch(imageDataAlpha._desc, imageDataBeta._desc, 2)

    matched1 = []
    matched2 = []
    nn_match_ratio = 0.8
    for m, n in nn_matches:
        if m.distance < nn_match_ratio * n.distance:
            matched1.append(imageDataAlpha._key[m.queryIdx])
            matched2.append(imageDataBeta._key[m.trainIdx])

    inliers1 = []
    inliers2 = []
    good_matches = []
    inlier_threshold = 2.5
    for i, m in enumerate(matched1):
        col = np.ones((3,1), dtype=np.float64)
        col[0:2,0] = m.pt
        col = np.dot(homography, col)
        col /= col[2,0]
        dist = sqrt(pow(col[0,0] - matched2[i].pt[0], 2) +
                    pow(col[1,0] - matched2[i].pt[1], 2))
        if dist < inlier_threshold:
            good_matches.append(cv2.DMatch(len(inliers1), len(inliers2), 0))
            inliers1.append(matched1[i])
            inliers2.append(matched2[i])

    if len(matched1) != 0 :
      inlier_ratio = len(inliers1) / float(len(matched1))
    else:
      inlier_ratio = 0

    if (inlier_ratio > 0.75):
      self.matches.append((imageDataAlpha, imageDataBeta))

    if (self.debug):
      print(imageName1 + "\t|\t" + imageName2 + "\t|\t" + str(inlier_ratio))
      if(self.debug):
        print('============= ' + imageName1 + ' ======= ' + imageName2 + ' ========' )
        print('Keypoints first image : \t', len(key_points1))
        print('Keypoints second image: \t', len(key_points2))
        print('Matches: \t', len(matched1))
        print('Inliers: \t', len(inliers1))
        print('Inliers Ratio: \t', inlier_ratio) 

# ***Klasa ZernikeImageMatcher***

In [18]:
class ZernikeImageMatcher:

  def __init__(self, groupOne, groupTwo, groupThree, debug):
    self.debug = debug
    self.groupOne = groupOne
    self.groupTwo = groupTwo
    self.groupThree = groupThree
    self.matches = []

  def run(self, debug):
    print("\nPorównywanie obrazów")
    print("\tGrupa 1")
    self.compareDescImageList(debug, self.groupOne)
    print("\tGrupa 2")
    self.compareDescImageList(debug, self.groupTwo)
    print("\tGrupa 3")
    self.compareDescImageList(debug, self.groupThree)

    return self.matches

  def genGroupToCompare(self, group):
    listAlpha = []
    listBeta = []

    for imageData in group:

      if imageData.getKind() == "Alpha":
        listAlpha.append(imageData)

      if imageData.getKind() == "Beta":
        listBeta.append(imageData)

    return listAlpha, listBeta 

  def compareDescImageList(self, debug, group):
    listAlpha, listBeta = self.genGroupToCompare(group)
    for imageDataAlpha in listAlpha:
      for imageDataBeta in listBeta:
          self.compareDescImage(imageDataAlpha, imageDataBeta, debug)
    
  def compareDescImage(self, imageDataAlpha, imageDataBeta, debug):
    distance = dist.euclidean(imageDataAlpha._desc, imageDataBeta._desc)
    print(imageDataAlpha._name + "\t|\t" + imageDataBeta._name + "\t|\t" + str(distance))
    if (float(distance) <= float(0.8) or distance == 0.0) :
      self.matches.append((imageDataAlpha, imageDataBeta))



# Klasa ***PrinterMatch***

In [19]:
class PrinterMatches:
  
  def __init__(self, listMatches):
      self.listMatches = listMatches

  def run(self):
    self.genMatchesDict()
    self.showResult(True)

  def genMatchesDict(self):
    result = {}
    for i in self.listMatches:  
      result.setdefault(i[0],[]).append(i[1])
    self.matchesDict = result

  def showResult(self, printResult):
    if (printResult):
      for query, listMachted in self.matchesDict.items():
        self.printMatch(query, listMachted)
        # self.showMatch(query, listMachted)
     
  def printMatch(self, query, listMachted):
    print("\n\tQuery: " + query._name)
    
    for imageData in listMachted:
      print("\t\t"+ imageData._name)

  def showMatch(self, query, listMachted):
    query.setName("Query: " + query._name)
    imagePrinter = ImagePrinter(listMachted)
    imagePrinter.showMatches(query)
  

# ***ImageManagerWithPreClassification***

In [20]:
class DescImagesManagerWithClassification:
  def __init__(self, imageListAlpha, imageListBeta, debug, ifPrintTwoGroup, ifPrintThreeGroup, descriptor):
    self.debug = debug
    self.descriptor = descriptor
    self.ifPrintTwoGroup = ifPrintTwoGroup
    self.ifPrintThreeGroup =ifPrintThreeGroup
    self.imageListAlpha = imageListAlpha
    self.imageListBeta = imageListBeta

    self.groupOne = []
    self.groupTwo = []
    self.groupThree = []

    self.matches = []
    self.matchesDict = {}

# ==========================================================
#                  Run
# ==========================================================
  def run(self):
    self.printOnBeforeStatus();
    timer = Timer()
    timer.start() 

    # Etap 1 wstepnej klasyfikacji - tworzenie grup
    self.generateGroup()
    # betaImagePrinter.showHistogramByMedium()

    # Etap 2 Generowanie cech lokalnych obrazów
    self.generateDescImageList()

    # Etap 3 Matching 
    self.compareDescImages(False)
    
    timer.stop()
    timer.printElapsedTime()

    # Etap 3 wydruk wyników
    self.printMatches()

# ==========================================================

  def printOnBeforeStatus(self):
    print("Wyszukiwanie obrazów podobnych w 2 zbiorach")
    print("\tZbiór alpha liczy {} obrazów".format(self.imageListAlpha.getSize()))
    print("\tZbiór beta liczy {} obrazów".format(self.imageListBeta.getSize()))
    print("\tW sumie {} obrazów".format(self.imageListAlpha.getSize() + self.imageListBeta.getSize()))

  def showGroups(self):
    oneImagePrinter = ImagePrinter(self.groupOne);
    oneImagePrinter.showImageList()
    
    twoImagePrinter = ImagePrinter(self.groupTwo);
    twoImagePrinter.showImageList()

    threeImagePrinter = ImagePrinter(self.groupThree);
    threeImagePrinter.showImageList()

# ==========================================================
#      Generowanie Grup
# ==========================================================

  def generateGroup(self):
    preClassificator = PreClassificator(alphaListImageData, betaListImageData, 14, self.ifPrintTwoGroup, self.ifPrintThreeGroup)
    self.groupOne, self.groupTwo, self.groupThree = preClassificator.run()


# ==========================================================
#      Generowanie Cech lokalnych obrazów
# ==========================================================

  def generateDescImageList(self):
    if (self.descriptor == "Akaze"):
      generetorDescriptor = GeneretorDescriptor(alphaListImageData, betaListImageData, True)
      self.imageListAlpha, self.imageListBeta = generetorDescriptor.run()
    if (self.descriptor == "Zernike"):
      zernikeGenereatorDescriptor = ZernikeGenereatorDescriptor(alphaListImageData, betaListImageData, True)
      self.imageListAlpha, self.imageListBeta = zernikeGenereatorDescriptor.run()

# ==========================================================
#      Wyszukiwanie obrazów podobnych
# ==========================================================

  def compareDescImages(self, debug):
    if (self.descriptor == "Akaze"):
      imageMatcher = ImageMatcher(self.groupOne, self.groupTwo, self.groupThree, False)
      self.matches = imageMatcher.run(False)
    if (self.descriptor == "Zernike"):
      imageMatcher = ZernikeImageMatcher(self.groupOne, self.groupTwo, self.groupThree, False)
      self.matches = imageMatcher.run(False)


# ==========================================================
#      Wyświetl dopasowane obrazy
# ==========================================================
  
  def printMatches(self):
    printerMatches = PrinterMatches(self.matches)
    printerMatches.run()
    

# ***ImageManager***
Bez wstępnej klasyfikacji

In [21]:
class ImageManager:
  def __init__(self, imageListAlpha, imageListBeta, debug, descriptor):
    self.debug = debug
    self.descriptor = descriptor
    self.imageListAlpha = imageListAlpha
    self.imageListBeta = imageListBeta

    self.matches = []
    self.matchesDict = {}

# ==========================================================
#                  Run
# ==========================================================
  def run(self):
    self.printOnBeforeStatus();
    timer = Timer()
    timer.start() 

    # Etap 1 Generowanie cech lokalnych obrazów
    self.generateDescImageList()

    # Etap 2 Matching 
    self.compareDescImages(False)
    
    timer.stop()
    timer.printElapsedTime()

    # Etap 3 wydruk wyników
    self.printMatches()

# ==========================================================

  def printOnBeforeStatus(self):
    print("Wyszukiwanie obrazów podobnych w 2 zbiorach")
    print("\tZbiór alpha liczy {} obrazów".format(self.imageListAlpha.getSize()))
    print("\tZbiór beta liczy {} obrazów".format(self.imageListBeta.getSize()))
    print("\tW sumie {} obrazów".format(self.imageListAlpha.getSize() + self.imageListBeta.getSize()))

# ==========================================================
#      Generowanie Cech lokalnych obrazów
# ==========================================================

  def generateDescImageList(self):
    if (self.descriptor == "Akaze"):
      generetorDescriptor = GeneretorDescriptor(alphaListImageData, betaListImageData, True)
      self.imageListAlpha, self.imageListBeta = generetorDescriptor.run()
    if (self.descriptor == "Zernike"):
      zernikeGenereatorDescriptor = ZernikeGenereatorDescriptor(alphaListImageData, betaListImageData, True)
      self.imageListAlpha, self.imageListBeta = zernikeGenereatorDescriptor.run()

# ==========================================================
#      Wyszukiwanie obrazów podobnych
# ==========================================================

  def compareDescImages(self, debug):
    if (self.descriptor == "Akaze"):
      imageMatcherBeta = ImageMatcherBeta(self.imageListAlpha, self.imageListBeta, False)
      self.matches = imageMatcherBeta.run(False)
    # if (self.descriptor == "Zernike"):
    #   imageMatcherBeta = ZernikeImageMatcher(self.groupOne, self.groupTwo, self.groupThree, False)
    #   self.matches = imageMatcher.run(False)


# ==========================================================
#      Wyświetl dopasowane obrazy
# ==========================================================
  
  def printMatches(self):
    printerMatches = PrinterMatches(self.matches)
    printerMatches.run()
    

# ***ImageMatcherBeta***

In [22]:
class ImageMatcherBeta:

  def __init__(self, groupOne, groupTwo, debug):
    self.debug = debug
    self.groupOne = groupOne.getImagesData()
    self.groupTwo = groupTwo.getImagesData()
    self.matches = []

  def run(self, debug):
    print("\nPorównywanie obrazów")
    self.compareDescImageList(debug)

    return self.matches

  def compareDescImageList(self, debug):
    countOperation = 0;
    for imageDataAlpha in self.groupOne:
      for imageDataBeta in self.groupTwo:
          self.compareDescImage(imageDataAlpha, imageDataBeta, debug)
          countOperation = countOperation + 1;

    print("\t|\tIlość operacji: " + str(countOperation))
    return countOperation


  def compareDescImage(self, imageDataAlpha, imageDataBeta, debug):
    homography = 7.6285898e-01
    matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_BRUTEFORCE_HAMMING)
    nn_matches = matcher.knnMatch(imageDataAlpha._desc, imageDataBeta._desc, 2)

    matched1 = []
    matched2 = []
    nn_match_ratio = 0.8
    for m, n in nn_matches:
        if m.distance < nn_match_ratio * n.distance:
            matched1.append(imageDataAlpha._key[m.queryIdx])
            matched2.append(imageDataBeta._key[m.trainIdx])

    inliers1 = []
    inliers2 = []
    good_matches = []
    inlier_threshold = 2.5
    for i, m in enumerate(matched1):
        col = np.ones((3,1), dtype=np.float64)
        col[0:2,0] = m.pt
        col = np.dot(homography, col)
        col /= col[2,0]
        dist = sqrt(pow(col[0,0] - matched2[i].pt[0], 2) +
                    pow(col[1,0] - matched2[i].pt[1], 2))
        if dist < inlier_threshold:
            good_matches.append(cv2.DMatch(len(inliers1), len(inliers2), 0))
            inliers1.append(matched1[i])
            inliers2.append(matched2[i])

    if len(matched1) != 0 :
      inlier_ratio = len(inliers1) / float(len(matched1))
    else:
      inlier_ratio = 0 

    if (inlier_ratio > 0.75):
      self.matches.append((imageDataAlpha, imageDataBeta))

    if (self.debug):
      print(imageName1 + "\t|\t" + imageName2 + "\t|\t" + str(inlier_ratio))
      if(self.debug):
        print('============= ' + imageName1 + ' ======= ' + imageName2 + ' ========' )
        print('Keypoints first image : \t', len(key_points1))
        print('Keypoints second image: \t', len(key_points2))
        print('Matches: \t', len(matched1))
        print('Inliers: \t', len(inliers1))
        print('Inliers Ratio: \t', inlier_ratio) 

# ***Main***
Uruchom program zgodnie z konfiguracją

In [ ]:
alphaListImageData = ListImageData(ALPHA_KIND, ListfilesAlpha)
betaListImageData = ListImageData(BETA_KIND, ListfilesBeta)

if USE_PRE_CLASSIFICATION:
  descImagesManagerWithClassification = DescImagesManagerWithClassification(alphaListImageData, betaListImageData, DEBUG, ifPrintTwoGroup, ifPrintThreeGroup, Descriptor)
  descImagesManagerWithClassification.run()
else:
  imageManager = ImageManager(alphaListImageData, betaListImageData, DEBUG, Descriptor) 
  imageManager.run()

# ***Analiza 1. bazy obrazów***
1.Analiza z wykorzystaniem wstępnej klasyfikacji

2.Analiza bez wykorzystania wstępnej klasyfikacji

In [ ]:
#Z wstepna klasyfikacja

ListfilesAlpha = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE1/ALPHA/*.jpg");
ListfilesBeta  = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE1/BETA/*.jpg");

for i in range(20):
  print("Start iteration: " + str(i))
  alphaListImageData = ListImageData(ALPHA_KIND, ListfilesAlpha)
  betaListImageData = ListImageData(BETA_KIND, ListfilesBeta)

  descImagesManagerWithClassification = DescImagesManagerWithClassification(alphaListImageData, betaListImageData, DEBUG, ifPrintTwoGroup, ifPrintThreeGroup, Descriptor)
  descImagesManagerWithClassification.run()
  print("Stop iteration: " + str(i))

In [ ]:
#Bez wstępnej klasyfikacji

ListfilesAlpha = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE1/ALPHA/*.jpg");
ListfilesBeta  = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE1/ALPHA/*.jpg");

for i in range(20):
  print("Start iteration: " + str(i))
  alphaListImageData = ListImageData(ALPHA_KIND, ListfilesAlpha)
  betaListImageData = ListImageData(BETA_KIND, ListfilesBeta)

  imageManager = ImageManager(alphaListImageData, betaListImageData, DEBUG, Descriptor)
  imageManager.run()
  print("Stop iteration: " + str(i))

# **Analiza 2. bazy obrazów**
1.Analiza z wykorzystaniem wstępnej klasyfikacji

2.Analiza bez wykorzystania wstępnej klasyfikacji

In [ ]:
#Z wstepna klasyfikacja

ListfilesAlpha = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE2/ALPHA/*.jpg");
ListfilesBeta  = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE2/BETA/*.jpg");

for i in range(20):
  print("Start iteration: " + str(i))
  alphaListImageData = ListImageData(ALPHA_KIND, ListfilesAlpha)
  betaListImageData = ListImageData(BETA_KIND, ListfilesBeta)

  descImagesManagerWithClassification = DescImagesManagerWithClassification(alphaListImageData, betaListImageData, DEBUG, ifPrintTwoGroup, ifPrintThreeGroup, Descriptor)
  descImagesManagerWithClassification.run()
  print("Stop iteration: " + str(i))

In [ ]:
#Bez wstępnej klasyfikacji

ListfilesAlpha = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE2/ALPHA/*.jpg");
ListfilesBeta  = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE2/BETA/*.jpg");

for i in range(20):
  print("Start iteration: " + str(i))
  alphaListImageData = ListImageData(ALPHA_KIND, ListfilesAlpha)
  betaListImageData = ListImageData(BETA_KIND, ListfilesBeta)

  imageManager = ImageManager(alphaListImageData, betaListImageData, DEBUG, Descriptor)
  imageManager.run()
  print("Stop iteration: " + str(i))

# **Analiza 3. bazy obrazów**
1.Analiza z wykorzystaniem wstępnej klasyfikacji

2.Analiza bez wykorzystania wstępnej klasyfikacji

In [ ]:
#Z wstepna klasyfikacja
ListfilesAlpha = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE3/ALPHA/*.jpg");
ListfilesBeta  = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE3/ALPHA/*.jpg");

for i in range(20):
  print("Start iteration: " + str(i))
  alphaListImageData = ListImageData(ALPHA_KIND, ListfilesAlpha)
  betaListImageData = ListImageData(BETA_KIND, ListfilesBeta)

  descImagesManagerWithClassification = DescImagesManagerWithClassification(alphaListImageData, betaListImageData, DEBUG, ifPrintTwoGroup, ifPrintThreeGroup, Descriptor)
  descImagesManagerWithClassification.run()
  print("Stop iteration: " + str(i))

In [ ]:
#Bez wstępnej klasyfikacji
ListfilesAlpha = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE3/ALPHA/*.jpg");
ListfilesBeta  = glob.glob ("/content/drive/MyDrive/Praca Magisterska/Shared/Images/FOTO/DATABASE3/ALPHA/*.jpg");

for i in range(20):
  print("Start iteration: " + str(i))
  alphaListImageData = ListImageData(ALPHA_KIND, ListfilesAlpha)
  betaListImageData = ListImageData(BETA_KIND, ListfilesBeta)

  imageManager = ImageManager(alphaListImageData, betaListImageData, DEBUG, Descriptor)
  imageManager.run()
  print("Stop iteration: " + str(i))